In [87]:
import pickle
import matplotlib.pyplot as plt
import pandas as pd
from nltk.stem import WordNetLemmatizer as Lemmatizer
import numpy as np
import re

In [3]:
with open('data/text.p', 'rb') as file:
    text = pickle.load(file)
with open('data/target.p', 'rb') as file:
    target = pickle.load(file)
with open('data/predictor.p', 'rb') as file:
    pred = pickle.load(file)

In [23]:
with open('models/model_up.p', 'rb') as file:
    model_up = pickle.load(file)
with open('models/vectorizer.p', 'rb') as file:
    vec = pickle.load(file)

FileNotFoundError: [Errno 2] No such file or directory: 'models/vectorizer.p'

In [61]:
with open('train/fires_train.p', 'rb') as file:
    fires = pickle.load(file)

In [67]:
(pd.to_datetime('2017-12-31') - pd.to_datetime('2003-01-01')).days

5478

In [98]:
train_mask = (pred['status_date'] - pd.to_datetime('2003-01-01')).apply(lambda x: x.days) <= 5478

In [69]:
pred_train = pred[(pred['status_date'] - pd.to_datetime('2003-01-01')).apply(lambda x: x.days) <= 5478]

In [79]:
text_train = pred_train[['existing_use', 'proposed_use', 'description']].fillna('').astype(str).apply(lambda x: ' '.join(x), axis = 1)

In [85]:
def hasnumber(s):
    return bool(re.search(r'\d', s))

In [88]:
text_train = np.array([' '.join([Lemmatizer.lemmatize('n', word.lower()) for word in doc.split() if not hasnumber(word)]) for doc in text_train])

In [89]:
text_train

array(['family dwelling family dwelling renew for & for final inspection nan',
       'apartment apartment correct violation in report of physical inspection application no dec nan',
       'apartment apartment correct violation in report of physical inspection application no dec nan',
       ..., 'apartment apartment common area- re-roofingnan',
       'apartment apartment common area- re-roofingnan',
       'apartment apartment common area- re-roofingnan'], dtype='<U343')

In [4]:
with open('locations.p', 'rb') as file:
    locations = pickle.load(file)

In [57]:
with open('models/model_down.p', 'rb') as file:
    model = pickle.load(file)

In [101]:
X_text = vec.fit_transform(text_train)

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [92]:
pred_train['location'] = pred_train['address'].apply(lambda x: locations[x])

C:\Users\pengj\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [94]:
pred_trim = pred_train[['existing_stories', 'address', 'status_date', 'location']]

In [104]:
target_train = target[train_mask.values]

In [106]:
pred_trim['has_existing_stories'] = pred_trim['existing_stories']>0

C:\Users\pengj\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [107]:
model_up.fit(X_text, pred_trim, target_train)

C:\Users\pengj\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [109]:
y_test = (target['fire_rate_after'].iloc[~train_mask.values] > target['fire_rate_before'].iloc[~train_mask.values]).astype(int)

In [118]:
X_text_test = vec.transform(text_test)

In [120]:
pred_trim = pred[['existing_stories', 'address', 'status_date', 'location']]

In [121]:
pred_trim_test = pred_trim[~train_mask.values]

In [126]:
pred_trim_test['has_existing_stories'] = (pred_trim_test['existing_stories']>0).astype(int)
pred_trim_test['existing_stories'].fillna(0, inplace = True)

C:\Users\pengj\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\pengj\Anaconda2\lib\site-packages\pandas\core\generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [55]:
from sklearn.metrics import roc_auc_score

In [127]:
roc_auc_score(y_test, model_up.predict_proba(pred_trim_test, X_text_test))

0.8429165983004447

In [ ]:
model_up.predict(pred_trim, text.reshape(-1,1))